In [1]:
from startup import *
from scipy.spatial.distance import squareform, pdist

In [42]:
columns = ['agree_id', 'content', 'class']

In [2]:
df = pd.read_csv('data/train.csv', encoding='utf-8')
df.columns = map(lambda x: x.lower(), df.columns)
df = df[df['proc_type_path'].notnull()]
df = df[df['content'].notnull()]
df = df[df['class'] != u'剔除']
df['content'] = u'类型: ' + df['proc_type_path'] + u'号码: ' + df['obstacle_servnbr'] + ' ' + df['content']

In [3]:
stat = df.groupby('proc_type_path')['agree_id'].count()
path_to_id = {row['proc_type_path']: i for i, row in stat.reset_index().iterrows()}
df['partition'] = df['proc_type_path'].apply(lambda x: path_to_id[x])
df['flag'] = 1

In [7]:
partition_min_size = 50000
while True:
    pivot = df.pivot_table(index='partition', columns='class', values='flag', aggfunc=sum, fill_value=0)
    print 'len(pivot)=%d :' % len(pivot),
    count = pivot.sum(axis=1)
    print 'count.min()=%d :' % count.min(),
    if count.min() >= partition_min_size:
        break
    weights = pivot.div(count, axis=0)
    distance_matrix = pd.DataFrame(
        squareform(pdist(weights)), columns=weights.index.unique(), index=weights.index.unique())
    sub_idx = count.idxmin()
    distance = distance_matrix[sub_idx]
    main_idx = distance[distance.index != sub_idx].idxmin()
    print sub_idx, '-->', main_idx
    df['partition'] = df['partition'].apply(lambda x: main_idx if x == sub_idx else x)

In [10]:
pivot = df.pivot_table(index='partition', columns='class', values='flag', aggfunc=sum, fill_value=0)
count = pivot.sum(axis=1)
weights = pivot.div(count, axis=0)

In [58]:
for i, p in df['partition'].drop_duplicates().reset_index(drop=True).iteritems():
    partition_data = df[df['partition'] == p].reset_index(drop=True)[columns]
    path = 'data/%s' % str(i)
    if not os.path.exists(path):
        print 'Mkdir %s ... ' % path,
        os.makedirs(path)
        print '(ok)'
    partition_data.to_csv('%s/train.csv' % path, encoding='utf-8', index=False)

Mkdir data/0 ...  (ok)
Mkdir data/1 ...  (ok)
Mkdir data/2 ...  (ok)
Mkdir data/3 ...  (ok)
Mkdir data/4 ...  (ok)
Mkdir data/5 ...  (ok)
Mkdir data/6 ...  (ok)
Mkdir data/7 ...  (ok)
Mkdir data/8 ...  (ok)
Mkdir data/9 ...  (ok)
Mkdir data/10 ...  (ok)
